<a href="https://colab.research.google.com/github/pranshurastogi29/Post-analysis-and-Suggestion-Engine/blob/main/Post_analysis_and_Suggestion_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U sentence-transformers
!pip install torch==1.7.0
!pip install nltk==3.4.5
!pip install colorama==0.4.4
!pip install transformers==3.4.0
!pip install -q streamlit

     |████████████████████████████████| 81kB 8.7MB/s 
     |████████████████████████████████| 2.1MB 34.8MB/s 
     |████████████████████████████████| 1.2MB 54.1MB/s 
     |████████████████████████████████| 901kB 47.8MB/s 
     |████████████████████████████████| 3.3MB 52.9MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.1.0-cp37-none-any.whl size=119615 sha256=5e421e6c17a24fafc6a58904f0c2948a94cdd277f4eac9b27082fb5ac6c2786a
  Stored in directory: /root/.cache/pip/wheels/84/cb/21/1066bff3027215c760ca14a198f698bca8fccb92e33e2327eb
Successfully built sentence-transformers
     |████████████████████████████████| 776.8MB 22kB/s 
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.7.0 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.0 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torc

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2021-04-22 17:22:12--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.6.97.115, 52.45.2.52, 52.204.93.39, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.6.97.115|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13828408 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  30.5MB/s    in 0.4s    

2021-04-22 17:22:13 (30.5 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13828408/13828408]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
%%writefile app.py
import streamlit as st
from transformers import pipeline , GPT2LMHeadModel ,GPT2Tokenizer
import time
import pandas as pd
import numpy as np
import joblib
import lightgbm as lgb
import re
from run_pplm import run_pplm_example , generate , load_language_model
import nltk
from collections import Counter
from sentence_transformers import SentenceTransformer

@st.cache(allow_output_mutation=True,suppress_st_warning=True)
def gpt2_generation_load():
    print('loading GPT2_model')
    model , tokenizer = load_language_model('/content/drive/My Drive/eluvio_data/trained_model/')
    print('GPT2_model loaded')
    return model , tokenizer

@st.cache(allow_output_mutation=True,suppress_st_warning=True)
def load_sentiment_model():
    print('loading sentiment model')
    sentiment_pipe = pipeline('sentiment-analysis')
    print('sentiment model loading')
    return sentiment_pipe

@st.cache(allow_output_mutation=True,suppress_st_warning=True)
def load_votes_model():
    print('loading votes model')
    embed = SentenceTransformer('distilbert-base-nli-mean-tokens')
    model_lgbm = lgb.Booster(model_file='/content/drive/My Drive/eluvio_data/3_model.txt')
    print('model loaded')
    return embed , model_lgbm

@st.cache(allow_output_mutation=True,suppress_st_warning=True)
def load_create_post_model(pretrained_model='gpt2-medium'):
    print('post_create_model_loading')
    model = GPT2LMHeadModel.from_pretrained(
        pretrained_model,
        output_hidden_states=True)
    # load tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model)
    print('post_create_model_loaded')
    return model , tokenizer

def get_sentiment(text, sentiment_pipe):
  sent = sentiment_pipe(text)[0]
  if sent['score'] < .55:
    sent['label'] = 'Neutral'
  return sent


def create_post_with_upvotes(text, model, tokenizer,class_label='0to20',bag_of_words='politics'):
  output = run_pplm_example(
    cond_text=text,
    model = model, 
    tokenizer=tokenizer,
    num_samples=3,
    bag_of_words=bag_of_words,
    discrim='generic',
    discrim_weights = '/content/drive/My Drive/eluvio_data/generic_classifier_head_epoch_9.pt',
    class_label=class_label,
    length=80,
    stepsize=0.03,
    sample=True,
    num_iterations=5,
    gamma=1.5,
    gm_scale=0.95,
    kl_scale=0.02,
    verbosity='regular'
  )
  return (output[1][1][13:] , output[1][2][13:] , output[2][1][13:])


def get_range_of_upvotes(text, embed, model_lgbm):
  embeddings = embed.encode(text, show_progress_bar=True)
  x_test = pd.DataFrame(embeddings)
  preds = np.argmax(model_lgbm.predict(x_test))+np.random.randint(5)
  return preds


def get_forward_phrases(text,model,tokenizer):
  text = generate(n=3,prompt=text ,model=model,tokenizer=tokenizer,return_as_list=True,min_length=15,max_length=21,temperature=1)
  return text

embed, model_lgbm = load_votes_model()
sentiment_pipe = load_sentiment_model()
model , tokenizer = gpt2_generation_load()
gpt_model , gpt_tokenizer = load_create_post_model()

st.title('Post analysis and Suggestion Engine')

st.markdown('Here you can enter the text in first text box and can see some vital statistics like sentiment and range of upvotes it might get')

if st.checkbox('Start Predictive Keyboard'):
    st.write('uncheck the box if you are done')
    text = st.text_input('Enter Text:',key=0)
    texts = get_forward_phrases(text, model , tokenizer)
    while True:
      if len(texts)==3:
        text = st.sidebar.selectbox(label='text completion',options=texts) 
        break
      else:
        pass
    st.write('You choose: '+text)
else: pass

#text = st.text_area("Enter Text:", value='', height=None, max_chars=None, key=None)

st.subheader('Once done you can see the sentiment and upvotes range')

if st.button('Tell me Sentiment'):
    output = get_sentiment(text, sentiment_pipe)
    st.markdown(text)
    st.markdown('It is '+str(output['label'])+' with score of '+str(output['score']*100))
else: pass

if st.button('Get Upvotes Score'):
  preds = get_range_of_upvotes(text, embed, model_lgbm)

  class_dict = {'0':'0to20',
                '1':'20to50',
                '2':'50to100',
                '3':'100to250',
                '4':'250to500'}
  st.markdown(text)
  if str(preds) == '0':
     st.markdown('Oh you might see '+class_dict[str(preds)]+' upvotes')
  if str(preds) == '1':
     st.markdown('Nice you might get '+class_dict[str(preds)]+' upvotes')
  if str(preds) == '2':
     st.markdown('Great you might see '+class_dict[str(preds)]+' upvotes')
  if str(preds) == '3':
     st.markdown('Awesome you might collect '+class_dict[str(preds)]+' upvotes')
  if str(preds) == '4':
     st.markdown('Hurray! you might get '+class_dict[str(preds)]+' upvotes')
else: pass

st.write("""Here You can create a post with specified needs with seed text you enter in the box, like you want to create a post with politics context targeting
certain number of upvotes the you can select different subjects and desired upvotes from dropdown lists.
plz select subject from subject list and desired upvotes from upvote list then click on Create post button """)

text2 = st.text_area('Enter your seed text', value='', height=None, max_chars=None, key=None)
subject = st.sidebar.selectbox("Select Subject",['legal','military','monsters',
                                                 'politics','religion','science',
                                                 'space','technology'])
upvotes = st.sidebar.selectbox("Select Upvotes intented",['0to20','20to50','100to500','500to_above'])

if st.button('Create a post'):
  out = create_post_with_upvotes(text2, model=gpt_model, tokenizer=gpt_tokenizer, class_label=upvotes, bag_of_words=subject)
  st.markdown('Here is the suggestion with '+upvotes+' upvotes on '+subject+' subject')
  st.markdown(str(out[1]))
else: pass

Writing app.py


In [ ]:
get_ipython().system_raw('./ngrok http 8501 &')

In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go to the following URL: https://cc35e915fa6c.ngrok.io


In [ ]:
!streamlit run "app.py"


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.86.177.24:8501

2021-04-22 17:33:53.949813: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
loading votes model
2021-04-22 17:33:56.420 Load pretrained SentenceTransformer: distilbert-base-nli-mean-tokens
2021-04-22 17:33:56.420 Did not find folder distilbert-base-nli-mean-tokens
2021-04-22 17:33:56.420 Search model on server: http://sbert.net/models/distilbert-base-nli-mean-tokens.zip
2021-04-22 17:33:56.421 Load SentenceTransformer from folder: /root/.cache/torch/sentence_transformers/sbert.net_models_distilbert-base-nli-mean-tokens
2021-04-22 17:34:04.857 Use pytorch device: cuda
model loaded
loading sentiment model
loading sentiment model
sentiment model loading
sentiment model loading
loading GPT2_model
GPT2_model loaded
post_create_model_loading
post_create_model_loaded
Batches: 100% 1/1 [00: